# Extraktion
Mit diesem Programm können relevante Messdaten aus den Bildreihen extrahiert werden.

## Importierte Programmbibliotheken

In [ ]:
import cv2 as cv, numpy as np, matplotlib.pyplot as plt, math, csv
from numpy.polynomial import Polynomial
from datetime import datetime 
# import time
# from time import localtime, strftime, gmtime, sleep
# import math
# import os
from os import listdir
from os.path import isfile, join
# import matplotlib.pyplot as plt
from ellipse import LsqEllipse

In [ ]:
# Zeigt ein Bild in einem separaten Fenster an
def showImage(image, title='Bild'):
    cv.imshow(title, image)
    k = cv.waitKey(0)
    cv.destroyAllWindows()

# Ein Verzeichnis erstellen, falls es noch nicht existiert
def createDir (directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
        return True
    except OSError as e:
        print (f'Fehler: Konnte Verzeichnis {directory} nicht erstellen. Grund: {str(e)}')
    return False

# Helfermethode, um eine in Polarkoordinaten angegebene Gerade zu zeichnen
def calcLinePoints(line):
    if line is None:
        return None, None
    
    r, theta = getR(line), getTheta(line)
    a = np.cos(theta)
    b = np.sin(theta)
        
    x0 = a * r
    y0 = b * r
        
    x1 = int(x0 + 1500 * (-b))
    y1 = int(y0 + 1500 * (a))
    x2 = int(x0 - 1500 * (-b))
    y2 = int(y0 - 1500 * (a))
       
    return (x1, y1), (x2, y2)

def drawLine(image, line, color = (255), width = 1, lineType = cv.LINE_8):
    point1, point2 = calcLinePoints(line) 
    
    if point1 is None or point2 is None:
        return
    
    cv.line(image, point1, point2, color, width, lineType)

getR = lambda line: line[0][0]
getTheta = lambda line: line[0][1]
toIntPoint = lambda point: (int(round(point[0])), int(round(point[1])))
createImage = lambda sizeWithHeightFirst: np.zeros(sizeWithHeightFirst, np.uint8)

def getSecondsFromFileName(fileName):
    try:
        name = fileName.split('.')[0]
        nameParts = name.split('-')
        parts = nameParts[1].split('_')
        return int(parts[0]) * 60 * 60 + int(parts[1]) * 60 + int(parts[2]) + int(parts[3]) / 1000
    except:
        return -1

def getFrameIdFromFileName(fileName):
    try:
        name = fileName.split('.')[0]
        nameParts = name.split('-')
        return nameParts[0]
    except:
        return -1

def sortFrames(fileName):
    parts = fileName.split('-')

    try:
        return int(parts[0])
    except:
        return -1

dist = lambda a, b: math.sqrt(a**2 + b**2) 

getR = lambda line: line[0][0]
getTheta = lambda line: line[0][1]
toGrad = lambda rad: rad / np.pi * 180
toRad = lambda grad: grad * np.pi / 180

def toCartesianCoordinates(line):
    r, theta = getR(line), getTheta(line)
    return -1 / math.tan(theta), r / math.sin(theta) # Kann undefiniert werden

def toR(b, theta):
    return b * math.sin(theta) # Kann undefiniert werden

def createSlopeMask(sizeWithHeightFirst, line):
    mask = createImage(sizeWithHeightFirst)
    height = sizeWithHeightFirst[0]
    m, b = toCartesianCoordinates(line)
    points = np.array([[(0, b), ((height - b)/m, height), (0, height)]], np.int32)
    cv.fillPoly(mask, points, (255))
    return cv.bitwise_not(mask)

def createEveneMask(sizeWithHeightFirst, line):
    mask = createImage(sizeWithHeightFirst)
    height = sizeWithHeightFirst[0]
    width = sizeWithHeightFirst[1]
    
    m, b = toCartesianCoordinates(line)
    points = np.array([[(0, b), (width, b), (width, height)]], np.int32)
    cv.fillPoly(mask, points, (255))
    return cv.bitwise_not(mask)

def findLongestContour(imageEdge, minContourLength = 100, concatenate = False):
    contours, _ = cv.findContours(imageEdge, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    contourCount = len(contours)
    maxContourLength = minContourLength
    maxContour = None
    
    if contours is None:
        return None
    
    if not concatenate:
        for contour in contours:
            contourLength = len(contour)
            if contourLength > maxContourLength:
                maxContourLength = contourLength 
                maxContour = contour
    else:
        candidates = []

        for contour in contours:
            contourLength = len(contour)
            if contourLength > maxContourLength:
                candidates.append(contour)

        if len(candidates) > 1:
            print(f'{len(candidates)} Konturen gefunden, mindeslänge {minContourLength}')
            maxContour = np.concatenate(candidates, axis = 0)
        elif len(candidates) == 1:
            maxContour = candidates[0]
    
    return maxContour

def sortContoursByLength(contour):
    return -len(contour)

def findLongestContours(imageEdge, minContourLength = 100):
    contours, _ = cv.findContours(imageEdge, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    contourCount = len(contours)
    maxContours = []
    
    if contours is None:
        return maxContours
    
    for contour in contours:
        contourLength = len(contour)
        if contourLength > minContourLength:
            maxContours.append(contour)

    maxContours.sort(key = sortContoursByLength)

    return maxContours

def findContourPoints(contour):
    contourBounds = cv.boundingRect(contour)
    left, width = contourBounds[0], contourBounds[2]
    top, height = contourBounds[1], contourBounds[3]

    half = left + width/2
    
    points = np.zeros((height, 3))
    
    for i in range(height):
        points[i][1] = left + width
    
    for point in contour:
        i = point[0][1] - top
        r = point[0][0]
        
        points[i][0] = i + top
        
        if r < points[i][1] and r < half:
            points[i][1] = r
            
        if r > points[i][2] and r > half:
            points[i][2] = r
            
    return points

def findAllContourPoints(contour, removeDuplicatePoints = True):
    contourBounds = cv.boundingRect(contour)
    left, width = contourBounds[0], contourBounds[2]
    middle = left + width / 2

    pointsLeft = []
    pointsRight = []

    for point in contour:
        x, y = point[0][0], point[0][1]
        
        if x > middle:
            pointsRight.append((x, y))
        else:
            pointsLeft.append((x, y))

    if removeDuplicatePoints:                               
        pointsLeft = removeDuplicates(pointsLeft)
        pointsRight = removeDuplicates(pointsRight)

    pointsLeft = [np.array([p[0], p[1]], np.float) for p in pointsLeft]
    pointsRight = [np.array([p[0], p[1]], np.float) for p in pointsRight]

    return np.array(pointsLeft), np.array(pointsRight)

def contourToPoints(contour, removeDuplicatePoints = True):
    points = [(p[0][0], p[0][1]) for p in contour]

    if removeDuplicatePoints:                               
        points = removeDuplicates(points)

    return np.array([np.array([p[0], p[1]], np.float) for p in points])

def splitContourPoints(contourPoints):
    pointCount = len(contourPoints)
    contourLeft = np.zeros((pointCount, 2))
    contourRight = np.zeros((pointCount, 2)) 
    
    for i in range(pointCount):
        contourLeft[i][0], contourLeft[i][1] = contourPoints[i][1], contourPoints[i][0]
        contourRight[i][0], contourRight[i][1] = contourPoints[i][2], contourPoints[i][0]
        
    return contourLeft, contourRight

def drawEllipse(image, ellipse, color = (255), lineType = cv.LINE_8):
    center, radius1, radius2, angle = ellipse.as_parameters()
    cv.ellipse(image, toIntPoint(center), toIntPoint((radius1, radius2)), angle * (180 / np.pi), 0, 360, color, lineType = lineType)

def sliceFromEnd(points, maxPointCount):
    pointCount = len(points)
    rangeStart = max(pointCount - maxPointCount, 0)
    
    x = [points[i][0] for i in range(rangeStart, pointCount)]
    y = [points[i][1] for i in range(rangeStart, pointCount)]
        
    return x, y

def sliceFromEnd2(points, maxPointCount):
    pointCount = len(points)
    rangeStart = max(pointCount - maxPointCount, 0)
    
    newPoints = [points[i] for i in range(rangeStart, pointCount)]
        
    return newPoints

def pointsToContour(points):
    return [[(p[0], p[1])] for p in points]

def calcIntersection(polynom, line, window):
    m, b = toCartesianCoordinates(line)
    
    if m == 0:
        return (b, polynom(b))
    else:
        windowSize = abs(window[0] - window[1])
        
        while windowSize > 0.1:
            y1 = m * window[0] + b
            y2 = m * window[1] + b
        
            x1 = polynom(y1)
            x2 = polynom(y2)
            
            window = (x1, x2)
            
            if windowSize <  abs(window[0] - window[1]):
                print('Keine Konvergenz')
                return (-1, -1)
                
            windowSize = abs(window[0] - window[1])
        
        return (x1, y1)

def sizeOf(image, heightFirst = False):
    if heightFirst:
        return (image.shape[0], image.shape[1])
    else:
        return (image.shape[1], image.shape[0])

def removeDuplicates(myList):
    return list(dict.fromkeys(myList))

def calcEllipseIntersection(ellipse, line):
    center, rMax, rMin, angle = ellipse.as_parameters()
    m, bOrig = toCartesianCoordinates(line)

    cos = math.cos(angle)
    sin = math.sin(angle)
    sinCos = sin * cos
   
    b = bOrig + m * center[0] - center[1]
    
    u = rMin**2 * (cos**2 + 2 * m * sinCos + (m * sin)**2) + rMax**2 * (sin**2 - 2 * m * sinCos + (m * cos)**2)
    v = rMin**2 * 2 * b * (sinCos + m * sin**2) + rMax**2 * 2 * b * (-sinCos +  m * cos**2)
    w = (rMin * b * sin)**2 + (rMax * b * cos)**2 - (rMin * rMax)**2

    if v**2 < 4 * u * w:
        print('Kein Schnittpunkt')
        return None, None

    x1 = ( math.sqrt(v**2 - 4 * u * w) - v) / (2 * u) + center[0]
    y1 = m * x1 + bOrig

    x2 = (-math.sqrt(v**2 - 4 * u * w) - v) / (2 * u) + center[0]
    y2 = m * x2 + bOrig

    if x1 > x2:
        return (x2, y2), (x1, y1)
    else:
        return (x1, y1), (x2, y2)

def calcEllipseX(ellipse, y):
    center, rMax, rMin, angle = ellipse.as_parameters()
    cos = math.cos(angle)
    sin = math.sin(angle)
    sinCos = sin * cos
   
    u = (rMin * cos)**2 + (rMax * sin)**2
    v = 2 * (y - center[1]) * sinCos * (rMin**2 - rMax**2)
    w = (rMin * (y - center[1]) * sin)**2 + (rMax * (y - center[1]) * cos)**2 - (rMin*rMax)**2

    if v**2 < 4 * u * w:
        print('Kein Schnittpunkt in x')
        return -1, -1

    x1 = ( math.sqrt(v**2 - 4 * u * w) - v) / (2 * u) + center[0]
    x2 = (-math.sqrt(v**2 - 4 * u * w) - v) / (2 * u) + center[0]

    return x1, x2

def calcEllipseSlope(ellipse, point):
    center, rMax, rMin, angle = ellipse.as_parameters()
    cos = math.cos(angle)
    sin = math.sin(angle)

    slopeRotated = -((((point[0] - center[0]) * cos + (point[1] - center[1]) * sin) /
        ((point[1] - center[1]) * cos - (point[0] - center[0]) * sin))) * (rMin / rMax)**2

    return math.tan(math.atan(slopeRotated) + angle)

def getNewB(line, dX, dY):
    m, b = toCartesianCoordinates(line)
    return b - dY + m * dX

copyLine = lambda line: [[line[0][0], line[0][1]]]

def splitPointCoords(points):
    x = np.array([p[0] for p in points])
    y = np.array([p[1] for p in points])
    
    return x, y

def calcAngleLeft(slope, line):
    surfaceAngle = np.pi / 2 - getTheta(line)
    return (np.pi if slope < 0 else 0) + math.atan(slope) + surfaceAngle

def calcAngleRight(slope, line):
    surfaceAngle = np.pi / 2 - getTheta(line)
    return (np.pi if slope > 0 else 0) - math.atan(slope) - surfaceAngle

def drawLine2(image, point, slope, color = (255), width = 1, lineType = cv.LINE_8):
    point1 = (point[0] + 1000, int(point[1] + 1000 * slope))
    point2 = (point[0] - 1000, int(point[1] - 1000 * slope))
    
    cv.line(image, point1, point2, color, width, lineType)

def calcCenterOfMass(imageBinary):
    reducedX = cv.reduce(imageBinary, 0, cv.REDUCE_SUM, dtype = cv.CV_32S)
    reducedY = cv.reduce(imageBinary, 1, cv.REDUCE_SUM, dtype = cv.CV_32S)

    x = np.array([r / 255 for r in reducedX[0]])
    y = np.array([r[0] / 255 for r in reducedY])

    countX = np.arange(1, len(x) + 1)
    countY = np.arange(1, len(y) + 1)

    centerX = np.sum(np.multiply(x, countX)) / np.sum(x)
    centerY = np.sum(np.multiply(y, countY)) / np.sum(y)

    return (centerX, centerY)

def findDropBounds(imageEdges):
    contour = findLongestContour(imageEdges, 50)

    if contour is not None:
         return cv.boundingRect(contour)
    else:
        return None

def getDistance(pointA, pointB):
    dX = pointB[0] - pointA[0]
    dY = pointB[1] - pointA[1]
    return (dX**2 + dY**2)**0.5

def findSpecificLine(lines, angle, maxAngleDelta):
    resultLine = None
    minAngleDelta = maxAngleDelta

    if lines is None:
        printt('Keine Linie mit ausreichender Länge gefunden')
    else:
        for line in lines:
            theta = getTheta(line)
            angleDelta = abs(theta - angle)

            if angleDelta < minAngleDelta:
                minAngleDelta = angleDelta
                resultLine = line 

        if resultLine is None:
            printt('Keine Linie mit geforderter Orientierung gefunden')

    return resultLine

In [ ]:
DIR_INDEX = 1
INSET = 0
directory = directories[DIR_INDEX]

resultDir = f'./Ergebnisse/V_{DIR_INDEX}'
createDir(resultDir)

mask = cv.imread(join(directory, "0-mask.bmp"))
mask = cv.cvtColor(mask, cv.COLOR_RGB2GRAY)

filePaths = [f for f in listdir(directory) if isfile(join(directory, f))]
filePaths.sort(key = sortFrames)

In [ ]:
directoriesPlane = [
    "Dummy",
    "E:/Bachelorarbeit/Sortiert/V1_Al2O3-L-1050/",
    "E:/Bachelorarbeit/Sortiert/V2_Al2O3-L-1100/",
    "E:/Bachelorarbeit/Sortiert/V3_Al2O3-L-1150/",
    "E:/Bachelorarbeit/Sortiert/V4_Al2O3-A-1050/",
    "E:/Bachelorarbeit/Sortiert/V5_Al2O3-A-1100/",
    "E:/Bachelorarbeit/Sortiert/V6_Al2O3-A-1150/",
    "E:/Bachelorarbeit/Sortiert/V7_GlasC-A-1050/",
    "E:/Bachelorarbeit/Sortiert/V8_GlasC-A-1100/",
    "E:/Bachelorarbeit/Sortiert/V9_GlasC-A-1150/",
    "E:/Bachelorarbeit/Sortiert/V10_PtAu-glatt-L-1100/",
    "E:/Bachelorarbeit/Sortiert/V11_PtAu-rau-L-1100/",
    "E:/Bachelorarbeit/Sortiert/V12_PtRh-glatt-L-1100/",
    "E:/Bachelorarbeit/Sortiert/V13_PtRh-rau-L-1100/"
]

linesPlane = [
    # Wird dynamisch bestimmt
]

insetsLeftPlane = [
    300, # 0
    300, # 1
    300, # 2
    300, # 3
    300, # 4
    300, # 4
    300, # 5
    300, # 5
    300, # 6
    300, # 7
    300, # 8
    300, # 9
    300, # 10
    300, # 11
    300, # 12
    300, # 13
]

insetsRightPlane = [
    300, # 0
    300, # 1
    300, # 2
    300, # 3
    300, # 4
    300, # 4
    300, # 5
    300, # 5
    300, # 6
    300, # 7
    300, # 8
    300, # 9
    300, # 10
    300, # 11
    300, # 12
    300, # 13
]

dropInsetsTopPlane = [
    10, # 0
    10, # 1
    10, # 2
    10, # 3
    10, # 4
    10, # 4
    10, # 5
    10, # 5
    10, # 6
    10, # 7
    10, # 8
    10, # 9
    10, # 10
    10, # 11
    10, # 12
    10, # 13
]

cannyThresholdsListPlane = [
    (60, 160), # 0
    (60, 160), # 1
    (60, 160), # 2
    (60, 160), # 3
    (60, 160), # 4
    (80, 160), # 5
    (40, 80),  # 6
    (80, 160), # 7
    (80, 160), # 8
    (80, 160), # 9
    (80, 160), # 10
    (80, 160), # 11
    (80, 160), # 12
    (80, 160)  # 13
]

dustCutoffsPlane = [
    0, # 0
    0, # 1
    0, # 2
    0, # 3
    0, # 4
    0, # 4
    0, # 5
    0, # 5
    0, # 6
    0, # 7
    0, # 8
    0, # 9
    0, # 10
    0, # 11
    0, # 12
    0, # 13
]

## Schräge

In [ ]:
## ECHTE PFADE
directoriesSlope = [
    "./Versuche/0_Al2O3_Luft_1100/",
    "./Versuche/1_C_Argon_1050/",
    "./Versuche/2_C_Argon_1100/",
    "./Versuche/3_C_Argon_1150/",
    "./Versuche/4_PtAu_Luft_1050/",
    "./Versuche/5_PtAu_Luft_1100/",
    "./Versuche/6_PtAu_Luft_1150/",
]

linesSlope = [
    [[10.0, 0.5]],    # 0
    [[510.0, 1.959]], # 1
    [[466.0, 1.967]], # 2
    [[459.0, 1.957]], # 3
    [[429.0, 1.969]], # 4
    [[404.0, 1.962]], # 5
    [[425.0, 1.966]], # 6
]

insetsLeftSlope = [
    10,  # 0
    100, # 1
    180, # 2 
    140, # 3
    0,   # 4
    0,   # 5
    10   # 6
]

insetsRightSlope = [
    10,  # 0
    250, # 1
    180, # 2
    240, # 3
    50,  # 4
    60,  # 5
    60   # 6
]

dropInsetsTopSlope = [
    10,   # 0
    10,   # 1
    10,   # 2
    10,   # 3
    -10,  # 4
    -10,  # 5
    -10   # 6
]

cannyThresholdsListSlope = [
    (70, 140),  # 0
    (80, 160),  # 1
    (90, 180),  # 2
    (90, 180),  # 3
    (100, 200), # 4
    (100, 200), # 5
    (100, 200)  # 6
]

dustCutoffsSlope = [
    2, # 0
    5, # 1
    3, # 2
    2, # 3
    2, # 4
    1, # 5
    0  # 6
]

In [ ]:
## ECHTE PFADE
directories = [directoriesPlane, directoriesSlope]
lines = [linesPlane, linesSlope]
insetsLeft = [insetsLeftPlane, insetsLeftSlope]
insetsRight = [insetsRightPlane, insetsRightSlope]
dropInsetsTop = [dropInsetsTopPlane, dropInsetsTopSlope]
cannyThresholdsList = [cannyThresholdsListPlane, cannyThresholdsListSlope]
dustCutoffs = [dustCutoffsPlane, dustCutoffsSlope]

In [ ]:
PX_TO_M = 3.45e-5

INSET_Y_TOP = 300  
INSET_Y_BO = 0 

MIN_CONTOUR_LENGTH = 60                  
POINT_COUNT = 80
POLYNOM_DEGREE = 2

USE_ELLIPSE = True
USE_POLYNOM = True 

VISUALIZE_CONTOURS = True
VISUALIZE_ELLIPSE = True
VISUALIZE_POLYNOM = True  

R_RES = 1
THETA_RES = np.pi / 360
LINE_MIN_LENGTH = 150

PLANE_ANGLE = np.pi / 2
PLANE_ANGLE_DELTA_MAX = np.pi * (3 / 180)

In [ ]:
def getPlaneEdge(mask):
    maskBlur = cv.GaussianBlur(mask, (3, 3), 0)
    _, maskBinary = cv.threshold(maskBlur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    maskEdges = cv.Canny(maskBinary, 255, 255, apertureSize = 3)

    lines = cv.HoughLines(maskEdges, R_RES, THETA_RES, LINE_MIN_LENGTH) # Probenkante ablesen
    line = findSpecificLine(lines, PLANE_ANGLE, PLANE_ANGLE_DELTA_MAX)
    return line
    
def getStartSeconds(directory):
    path = join(directory, "2 Kontakt")
    files = [f for f in listdir(path) if isfile(join(path, f))]
    files.sort(key = sortFrames)
    return getSecondsFromFileName(files[0])

def getMask(directory, isSloped):
    if isSloped:
        return cv.imread(join(directory, "0-mask.bmp"))
    else:
        path = join(directory, "1 Probe")
        files = [f for f in listdir(path) if isfile(join(path, f))]
        files.sort(key = sortFrames)
        return cv.imread(join(path, files[0]))

def getFilePaths(directory, isSloped):
    if isSloped:
        return [f for f in listdir(directory) if isfile(join(directory, f))]
    else:
        path = join(directory, "3 Spreitung")
        return [f for f in listdir(path) if isfile(join(path, f))]


DIR_INDEX = 2
IS_SLOPED = True
INSET = 0

experimentIndex = 1 if IS_SLOPED else 0
rotateImage = IS_SLOPED
calcCenter = IS_SLOPED
useOneEllipse = not IS_SLOPED

directory = directories[experimentIndex][DIR_INDEX]

experimentTag = 'schief' if IS_SLOPED else 'eben'
resultDir = f'./Dokument2/Auswertung/Neu/{experimentTag}/V{DIR_INDEX}'
createDir(resultDir) 
                                                             
mask = getMask(directory, IS_SLOPED)
mask = cv.cvtColor(mask, cv.COLOR_RGB2GRAY)

filePaths = getFilePaths(directory, IS_SLOPED)
filePaths.sort(key = sortFrames)

sizeImage = sizeOf(mask) 

insetLeft = insetsLeft[experimentIndex][DIR_INDEX]
insetRight = insetsRight[experimentIndex][DIR_INDEX]

maskCropped = mask[INSET_Y_TOP:960-INSET_Y_BO, insetLeft:(1280 - insetRight)]
croppedSizeHeightFirst = sizeOf(maskCropped, True)

croppedHeight = croppedSizeHeightFirst[0]
croppedWidth = croppedSizeHeightFirst[1]

edge = lines[1][DIR_INDEX] if IS_SLOPED else getPlaneEdge(maskCropped)
dropInsetTop = dropInsetsTop[experimentIndex][DIR_INDEX]
cannyThresholds = cannyThresholdsList[experimentIndex][DIR_INDEX]
dustCutoff = dustCutoffs[experimentIndex][DIR_INDEX]

if rotateImage:
    bNew = getNewB(edge, insetLeft, INSET_Y_TOP)
    rNew = toR(bNew, edge[0][1])
    shiftedEdge = [[rNew, edge[0][1]]]
else:
    bNew = 0
    rNew = 0
    shiftedEdge = edge

secondsList = []
fileNameList = []
centerOfMassList = []
eAngleLeftList = []
eAngleRightList = []
pAngleLeftList = []                                                                 
pAngleRightList = []
eIntersecDistList = []

startSeconds = 0

for i in range(1, len(filePaths)):                                
    fileName = filePaths[i]                                             
    seconds = getSecondsFromFileName(fileName)
    imageDir = directory if IS_SLOPED else join(directory, '3 Spreitung')          
    image = cv.imread(join(imageDir, fileName))

    if i == 1:
        startSeconds = seconds if IS_SLOPED else getStartSeconds(directory)

    image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    sizeImage = sizeOf(image)
    widthImage = sizeImage[0]
    heightImage = sizeImage[1]
                                                                                               
    imageCropped = image[INSET_Y_TOP:heightImage - INSET_Y_BO, insetLeft:(widthImage - insetRight)]
    imageDrop = cv.addWeighted(imageCropped, -1, maskCropped, 1, 0)
    
    newEdge = copyLine(shiftedEdge)

    if calcCenter:
        cropSlopeMask = createSlopeMask(croppedSizeHeightFirst, newEdge)

        _, imageBinary = cv.threshold(imageDrop, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        imageBinary = cv.bitwise_and(imageBinary, cropSlopeMask)
        centerOfMass = calcCenterOfMass(imageBinary)
    else:
        centerOfMass = (0, 0)

    if rotateImage:
        heightRotated = rNew + 100
        widthRotated = (croppedWidth**2 + croppedHeight**2)**0.5
        sizeRotated = toIntPoint((widthRotated, heightRotated))

        rotation = toGrad(getTheta(edge)) - 90
        matrix = cv.getRotationMatrix2D((0, 0), rotation, 1)

        imageDrop = cv.warpAffine(imageDrop, matrix, sizeRotated)
        newEdge[0][1] = 90/180 * np.pi
    else:
        sizeRotated = sizeOf(imageCropped)

    imageEdges = cv.Canny(imageDrop, cannyThresholds[0], cannyThresholds[1], apertureSize = 3)

    if rotateImage:
        cv.rectangle(imageEdges, (0, int(rNew) - dustCutoff), sizeRotated, (0), cv.FILLED)
    else:
        drawLine(imageEdges, edge, color = (0), width = 2)

    dropBounds = findDropBounds(imageEdges)

    upperCutoff = dropBounds[1] + dropInsetTop
    cv.rectangle(imageEdges, (0, upperCutoff), (sizeRotated[0], 0), (0), cv.FILLED)
                                                                                 
    contours = findLongestContours(imageEdges, minContourLength = MIN_CONTOUR_LENGTH)

    if len(contours) is 0:
        print(f'{fileName}: Keine Konturen im Bild gefunden!')
        continue

    if len(contours) == 1:
        contourLeft = contours[0]
        contourRight = None

        pointsLeft = contourToPoints(contourLeft)
        pointsRight = None
    else:
        contour0Left = cv.boundingRect(contours[0])[0]
        contour1Left = cv.boundingRect(contours[1])[0]

        if contour0Left > contour1Left:
            contourLeft = contours[1]
            contourRight = contours[0]
        else:
            contourLeft = contours[0]
            contourRight = contours[1]

        pointsLeft = contourToPoints(contourLeft)
        pointsLeftBackup = contourToPoints(contourLeft)
        pointsRight = contourToPoints(contourRight)

    if useOneEllipse and pointsRight is not None:
        pointsLeft = np.concatenate((pointsLeft, pointsRight))

    if VISUALIZE_CONTOURS or (USE_ELLIPSE and VISUALIZE_ELLIPSE) or (USE_POLYNOM and VISUALIZE_POLYNOM):
        imageResult = cv.bitwise_or(imageEdges, imageDrop)
        imageResult = cv.cvtColor(imageResult, cv.COLOR_GRAY2RGB)
        drawLine(imageResult, newEdge, (255, 255, 255))

        cv.drawContours(imageResult, [contourLeft], 0, (255, 0, 0))
        if contourRight is not None:
            cv.drawContours(imageResult, [contourRight], 0, (0, 0, 255))

    if USE_ELLIPSE:
        ellipseLeft = LsqEllipse().fit(pointsLeft)

        if not useOneEllipse and pointsRight is not None:
            ellipseRight = LsqEllipse().fit(pointsRight)
            eIntersecLeft, _ = calcEllipseIntersection(ellipseLeft, newEdge)
            _, eIntersecRight = calcEllipseIntersection(ellipseRight, newEdge)
        else:
            ellipseRight = None
            eIntersecLeft, eIntersecRight = calcEllipseIntersection(ellipseLeft, newEdge)
 
        if eIntersecLeft is None or eIntersecRight is None:
            print(f'{fileName}: Kein Ellipsen-Schnittpunkt gefunden')

            if VISUALIZE_ELLIPSE:
                drawEllipse(imageResult, ellipseLeft, color = (210, 130, 76))
                if ellipseRight is not None:
                    drawEllipse(imageResult, ellipseRight, color = (110, 67, 226))
                showImage(imageResult)
            
            continue

        eIntersecDist = PX_TO_M * getDistance(eIntersecLeft, eIntersecRight)
        ellipse = ellipseRight if ellipseRight is not None else ellipseLeft

        eSlopeLeft = calcEllipseSlope(ellipseLeft, eIntersecLeft)
        eSlopeRight = calcEllipseSlope(ellipse, eIntersecRight)

        eAngleLeft = np.pi - calcAngleLeft(eSlopeLeft, newEdge)
        eAngleRight = np.pi - calcAngleRight(eSlopeRight, newEdge)

        print(f'Winkel: {toGrad(eAngleLeft)} und {toGrad(eAngleRight)} (Ellipse)')

        if VISUALIZE_ELLIPSE:
            drawEllipse(imageResult, ellipseLeft, color = (210, 130, 76))
            if ellipseRight is not None:
                drawEllipse(imageResult, ellipseRight, color = (110, 67, 226))

            cv.circle(imageResult, toIntPoint(eIntersecLeft), 4, (255, 255, 255))
            cv.circle(imageResult, toIntPoint(eIntersecRight), 4, (255, 255, 255))

            drawLine2(imageResult, toIntPoint(eIntersecLeft), eSlopeLeft, color = (0, 230, 150))
            drawLine2(imageResult, toIntPoint(eIntersecRight), eSlopeRight, color = (0, 230, 150))
 
            cv.line(imageResult, toIntPoint((eIntersecLeft[0]+2, rNew)), toIntPoint((eIntersecRight[0] - 2, rNew)), (255, 255, 255))

            # Beschriftung
            text = f'{round(seconds - startSeconds, 2)}s - theta_r (links): {round(toGrad(eAngleLeft), 2)} - theta_a (rechts): {round(toGrad(eAngleRight), 2)}'
            cv.putText(imageResult, text, (40, int(rNew) + 80), cv.FONT_HERSHEY_SIMPLEX,  1, (255, 255, 255) , 2, cv.LINE_AA) 
    else:
        eAngleLeft = -1
        eAngleRight = -1
        eIntersecDist = -1

    if USE_POLYNOM and pointsRight is not None: # Annäherung allein durch ein Polynom ist sinnlos
        xLeft, yLeft = splitPointCoords(pointsLeftBackup)
        xRight, yRight = splitPointCoords(pointsRight)

        polynomLeft = Polynomial.fit(yLeft, xLeft, POLYNOM_DEGREE) #, w = pointWeightsLeft) # Vertauschte Achsen
        polynomRight = Polynomial.fit(yRight, xRight, POLYNOM_DEGREE) # , w = pointWeightsRight) # Vertauschte Achsen

        # Schnittpunkt
        widthImage = sizeOf(imageDrop)[0]
        pIntersecLeft = calcIntersection(polynomLeft, newEdge, (0, widthImage))
        pIntersecRight = calcIntersection(polynomRight, newEdge, (0, widthImage))

        pSlopeLeft = polynomLeft.deriv()(pIntersecLeft[1])
        pSlopeRight = polynomRight.deriv()(pIntersecRight[1])

        pAngleLeft = np.pi + calcAngleLeft(pSlopeLeft, newEdge) - np.pi / 2
        pAngleRight = np.pi + calcAngleRight(pSlopeRight, newEdge) - np.pi / 2

        print(f'Winkel: {toGrad(pAngleLeft)} und {toGrad(pAngleRight)} (Polynom)')

        if VISUALIZE_POLYNOM:
            polyPointsLeft = np.array([[polynomLeft(y), y] for y in range(0, sizeRotated[1])])
            polyPointsRight = np.array([[polynomRight(y), y] for y in range(0, sizeRotated[1])])

            # Polynome zeichnen
            cv.polylines(imageResult, np.int32([polyPointsLeft]), False, (255, 255, 0)) #, thickness = 1)
            cv.polylines(imageResult, np.int32([polyPointsRight]), False, (0, 0, 255))

            cv.circle(imageResult, toIntPoint(pIntersecLeft), 2, (255, 255, 255))
            cv.circle(imageResult, toIntPoint(pIntersecRight), 2, (255, 255, 255))

            drawLine2(imageResult, toIntPoint(pIntersecLeft), 1 / pSlopeLeft, color = (230, 0, 150))
            drawLine2(imageResult, toIntPoint(pIntersecRight), 1 / pSlopeRight, color = (230, 0, 150))
    else:
        pAngleLeft = -1
        pAngleRight = -1

    secondsList.append(seconds)
    fileNameList.append(fileName)
    centerOfMassList.append(centerOfMass)
    eAngleLeftList.append(eAngleLeft)  
    eAngleRightList.append(eAngleRight)
    pAngleLeftList.append(pAngleLeft)
    pAngleRightList.append(pAngleRight)
    eIntersecDistList.append(eIntersecDist)

    if VISUALIZE_CONTOURS or (USE_ELLIPSE and VISUALIZE_ELLIPSE) or (USE_POLYNOM and VISUALIZE_POLYNOM):
        name = fileName.split('.')[0]
        cv.imwrite(join(resultDir, f'{name}.png'), imageResult)
        pass                          

with open(join(resultDir, f'slope_ALL_V{DIR_INDEX}.csv'), 'w', newline='') as csvFile:
    datawriter = csv.writer(csvFile, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for i in range(len(secondsList)):
        datawriter.writerow([
            secondsList[i] - startSeconds, fileNameList[i], 
            centerOfMassList[i][0], centerOfMassList[i][1],
            eAngleLeftList[i], eAngleRightList[i], 
            pAngleLeftList[i], pAngleRightList[i],
            eIntersecDistList[i]
        ])

velocityList = [
    PX_TO_M * getDistance(centerOfMassList[i], centerOfMassList[i + 1]) / 
    (secondsList[i + 1] - secondsList[i])
    for i in range(len(centerOfMassList) - 1)
]

shiftedSecondsList = [(secondsList[i] + secondsList[i + 1]) / 2 for i in range(len(secondsList) - 1)]

with open(join(resultDir, f'slope_velocity_V{DIR_INDEX}.csv'), 'w', newline='') as csvFile:
    datawriter = csv.writer(csvFile, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for i in range(len(shiftedSecondsList)):
        datawriter.writerow([
            shiftedSecondsList[i] - startSeconds, 
            velocityList[i]
        ])

In [ ]:
def loadVelocities(index):
    resultDir = f'./Dokument2/Auswertung/schief/V{index}'

    secondsList = []                                        
    velocityList = []                                               
    listS = []

    with open(join(resultDir, f'slope_velocity_V{index}.csv'), newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for row in reader:
            secondsList.append(float(row[0]))
            velocityList.append(float(row[1]))

    return secondsList, velocityList

def loadEllipseAngles(index):
    resultDir = f'./Dokument2/Auswertung/schief/V{index}'

    secondsList = []                                        
    eAngleLeftList = []                                               
    eAngleRightList = []

    with open(join(resultDir, f'slope_ALL_V{index}.csv'), newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for row in reader:
            secondsList.append(float(row[0]))
            eAngleLeftList.append(toGrad(float(row[4])))
            eAngleRightList.append(toGrad(float(row[5])))

    return secondsList, eAngleLeftList, eAngleRightList

def loadContactDistances(index):
    resultDir = f'./Dokument2/Auswertung/schief/V{index}'

    secondsList = []                                        
    eContactDistList = []

    with open(join(resultDir, f'slope_ALL_V{index}.csv'), newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for row in reader:
            secondsList.append(float(row[0]))
            eContactDistList.append(float(row[8]))

    return secondsList, eContactDistList

## Glas-Kennwerte

In [ ]:
# Viskosität [Pa*s]
def getEta(T):
    ETA_A = -2.539
    ETA_B = 4126.91 # Prüfen
    ETA_T0 = 277.42
    return (10**(ETA_A + ETA_B / (T - ETA_T0))) * 10 # Pa * s

# Oberflächenspannung [N/m]
def getGamma(T):
    GAMMA_1400 = 316.98e-3  # N/m
    DELTA_GAMMA = -4e-3 # N/m pro 100 K
    return GAMMA_1400 + ((T - 1400) / 100) * DELTA_GAMMA

# Dichte [kg/m^3]
getRho = lambda T: 2367.0 + ((2328.0 - 2367.0) / (1400 - 1200)) * (T - 1200)

In [ ]:
# Mittlere Kontaktwinkel (links: R, rechts: A), in Grad
angleList =  [
    (123.75242724324829, 164.61359876269347),
    (120.78645851370071, 160.46087944487482),
    (146.5815616483129, 159.34435011342916),
    (48.95658941927519, 76.23038034900753),
    (48.888238143939, 75.74257109358932),
    (50.314125645102955, 76.97418912494616)
]

# Mittlere Kontaktlängen: DURCHMESSER [m]
contactDiameters = [
    0.005855576942737779,
    0.006249420817866661,
    0.005118851067856464,
    0.011158176843441593,
    0.011198394585055742,
    0.011088854190168968
]

# Massen [g]
m1 = 0.347
m2 = 0.355
m3 = 0.346 # Unsicher
m4 = 0.344	
m5 = 0.358 # Unsicher
m6 = 0.367

# Winkel der schiefen Ebene
a1 = 1.959 - np.pi/2
a2 = 1.967 - np.pi/2
a3 = 1.957 - np.pi/2

a4 = 1.969 - np.pi/2
a5 = 1.962 - np.pi/2
a6 = 1.966 - np.pi/2

def calcForceAdhesion(R, T, angles):
    gamma = getGamma(T)
    return (24 / np.pi**3) * gamma * 2*R * (math.cos(toRad(angles[0])) - math.cos(toRad(angles[1])))

def calcForceShearWrong(contactDia, T):
    eta = getEta(T)
    A_contact = np.pi * (contactDia / 2)**2
    dvdy = 1 # TODO: Das stimmt nicht!
    return eta * A_contact * dvdy 

calcDropRadius = lambda mass, density: ((3*mass)/(4*np.pi*density))**(1/3)

def calcTerminalVelocity(angle, R, T):
    return (math.sin(angle) * getGamma(T)**1.5) / (getEta(T) * R * (getRho(T) * 9.81)**0.5)

def calcOmega2(index, m, T, alpha):
    R = calcDropRadius(m, getRho(T))
    contactD = contactDiameters[index - 1]
    A_contact = np.pi * (contactD / 2)**2
    eta = getEta(T)
    g = 9.81
    c = 145

    F_g = m * g * math.sin(alpha)
    F_ad = calcForceAdhesion(R, T, angleList[index - 1])
    
    o = (2/5) * m * R
    p = A_contact * eta * c / o
    q = (-F_g + F_ad) / o

    result1 = -p/2 + ((p/2)**2 - q)**0.5
    result2 = -p/2 - ((p/2)**2 - q)**0.5

    return result1, result2

def calcOmega(index, m, T, alpha):
    R = calcDropRadius(m, getRho(T))
    contactD = contactDiameters[index - 1]
    preFactor = 5 / (2 * m * R)
    g = 9.81
    F_g = m * g * math.sin(alpha)
    F_ad = calcForceAdhesion(R, T, angleList[index - 1])
    F_tau = calcForceShear(contactD, T)
    return (preFactor * (F_g - F_ad - F_tau))**0.5


In [ ]:
for i in range (301, 681+1):
    print(i)


In [ ]:
R1 = calcDropRadius(m1 * 1e-3, getRho(1050))
R2 = calcDropRadius(m2 * 1e-3, getRho(1100))
R3 = calcDropRadius(m3 * 1e-3, getRho(1150))

R4 = calcDropRadius(m4 * 1e-3, getRho(1050))
R5 = calcDropRadius(m5 * 1e-3, getRho(1100))
R6 = calcDropRadius(m6 * 1e-3, getRho(1150))

print(R1*1e3)
print(R2*1e3)
print(R3*1e3)
print("-----")
print(R4*1e3)
print(R5*1e3)
print(R6*1e3)
print("-----")
vs1 = calcTerminalVelocity(a1, R1, 1050)
vs2 = calcTerminalVelocity(a2, R2, 1100)
vs3 = calcTerminalVelocity(a3, R3, 1150)

print(vs1)
print(vs2)
print(vs3)


print("Adhäsion")

F_ad = calcForceAdhesion(R1, 1050, angleList[1 - 1])
F_tau = calcForceShear(R1, 1050)
print(F_ad)

omega1 = calcOmega(1, m1, 1050, a1)
omega1a, omega1b = calcOmega2(1, m1, 1050, a1)
print(omega1*R1*1e4)
print(omega1a*R1*1e4, omega1b*R1*1e4)

omega2a, omega2b = calcOmega2(2, m2, 1100, a2)
print(omega2a*R2*1e4, omega2b*R2*1e4)

omega3a, omega3b = calcOmega2(3, m3, 1150, a3)
print(omega3a*R3*1e4, omega3b*R3*1e4)


omega4a, omega4b = calcOmega2(4, m4, 1050, a4)
print(omega4a*R4*1e4, omega4b*R4*1e4)


In [ ]:
3.2576160238993657
3.2869475066612104
3.263359731595706
-----
3.248200896179301
3.2961805537759314
3.328089053712699

In [ ]:
lineList = [
    22.242432066128288,
    22.700798302232926,
    22.127840507102107,
    22.815389861259106,
    22.41431940466751,
    22.643502522719857
]

n = len(lineList)
aMean = sum(lineList) / n

diffList = [a - aMean for a in lineList]
print(diffList)

print(aMean)

In [ ]:
toRad(180)

Geschwindigkeit durch Kräftegleichgewicht bestimmen

In [ ]:
for V in range(1, 6+1):
    _, velocityList = loadVelocities(V)
    n = len(velocityList)
    vMean = sum(velocityList) / n
    partSum = sum([(v - vMean)**2 for v in velocityList])
    stdAbw = (partSum / (n - 1))**0.5
    unsi = stdAbw / n**0.5

    print(f'v{V} Mean: {vMean}, std: {stdAbw}, unsi: {unsi}')

for V in range(1, 6+1):
    _, leftList, rightList = loadEllipseAngles(V)
    angleList = leftList

    # Links
    n = len(angleList)
    angleMean = sum(angleList) / n
    partSum = sum([(angle - angleMean)**2 for angle in angleList])
    stdAbw = (partSum / (n - 1))**0.5
    unsi = stdAbw / n**0.5

    print(f'v{V} left: Mean: {angleMean}, std: {stdAbw}, unsi: {unsi}')

for V in range(1, 6+1):
    _, leftList, rightList = loadEllipseAngles(V)
    angleList = rightList

    # Links
    n = len(angleList)
    angleMean = sum(angleList) / n
    partSum = sum([(angle - angleMean)**2 for angle in angleList])
    stdAbw = (partSum / (n - 1))**0.5
    unsi = stdAbw / n**0.5

    print(f'v{V} righ: Mean: {angleMean}, std: {stdAbw}, unsi: {unsi}')


for V in range(1, 6+1):
    _, lList = loadContactDistances(V)

    # Links
    n = len(lList)
    lMean = sum(lList) / n
    # partSum = sum([(angle - angleMean)**2 for angle in angleList])
    # stdAbw = (partSum / (n - 1))**0.5
    # unsi = stdAbw / n**0.5

    print(f'v{V} cdis: Mean: {lMean}')

In [ ]:
def toVelocity(pxToMm, x, y, t):
    return np.array([pxToMm * (dist(x[i + 1], y[i + 1]) - dist(x[i], y[i])) / (t[i + 1] - t[i]) for i in range(len(x) - 1)])


def loadExperimentOld(index):
    resultDir = f'./Ergebnisse/V_{index}'

    listX = []                                        
    listY = []                                               
    listS = []

    with open(join(resultDir, f'slope_V_{index}.csv'), newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for row in reader:
            listS.append(float(row[0]))
            listX.append(float(row[1]))
            listY.append(float(row[2]))

    return listS, listX, listY


def compareVelocities(experiments):
    velocities = []
    times = []

    for i in range(len(experiments)):
        experiment = experiments[i]

        listS, listX, listY = loadExperiment(experiment)
        print(listS[0])

        times.append(listS)
        velocities.append(toVelocity(3.45e-2, listX, listY, listS))

    
    colors = ['r', 'g', 'b', 'y', 'ro', 'go', 'bo', 'yo']

    print(f'Versuche: {experiments}')

    for i in range(len(velocities)):
        plt.plot(times[i][:-1], velocities[i], colors[i])
        
    plt.show()



In [ ]:
DIR_INDEX = 6

resultDir = f'./Ergebnisse/V_{DIR_INDEX}'

listX = []                                        
listY = []                                               
listS = []

with open(join(resultDir, f'slope_V_{DIR_INDEX}.csv'), newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

    for row in reader:
        listX.append(float(row[0]))
        listY.append(float(row[1]))
        listS.append(float(row[2]))

print(f'Daten zu Versuch {DIR_INDEX}')

plt.plot(listS, listX, 'g')
plt.plot(listS, listY, 'y')
plt.show()

PX_TO_MM = 3.45e-2

# Einheit mm/s
velocity = np.array([PX_TO_MM * (dist(listX[i + 1], listY[i + 1]) - dist(listX[i], listY[i])) / (listS[i + 1] - listS[i]) for i in range(len(listX) - 1)])
        

compareVelocities([1])

In [ ]:
def makeLevel(index, directory):
    listS, listX, listY = loadExperiment(index)
    s0 = listS[0]

    with open(join(directory, f'slope_V_{index}.csv'), 'w', newline='') as csvFile:
        datawriter = csv.writer(csvFile, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for i in range(len(listS)):
            datawriter.writerow([listS[i]-s0, listX[i], listY[i]])

